Author : Yassine El Ouahidi, inspired by Myriam Bontonou

Using the IBC dataset, we can define a classification problem where a class is a condition studied during a fMRI task (e.g. *audio sentence*, *speech sound*, *face gender*, *reward*, *left hand*...). The data samples are elementary contrast maps associated with a condition studied during a task performed by a subject. 

Broadly speaking, an elementary contrast map is a 3D volume. Each voxel (elementary volume) represents the activation of an area of the brain with respect to the studied condition.

In this notebook, we prepare a **benchmark** where the **classes** are **conditions** and the **data samples** are **elementary contrast maps**.

# Run this notebook to prepare the benchmark for your own work. 

In [1]:
## TO UPDATE ##
# Directory where the IBC dataset will be downloaded.
data_dir = '/users/local/nilearn_data/'

# Directory where the file Glasser_masker.nii.gz is stored.
path_Glasser = '/homes/y17eloua/Projets/spectral_conv_neuroimaging_classification/dataset/Glasser_masker.nii.gz'

collection_id = 4337

If you are interested, you can look at the steps we went through, from the downloading of the IBC dataset on NeuroVault to the split of the classes in a base dataset, a validation dataset and a novel dataset.

# 1. Download IBC dataset on NeuroVault.

In [2]:
import os
import numpy as np
from numpy import savez_compressed
from tqdm import tqdm
import pandas as pd
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from itertools import compress
import nibabel as nib
import nilearn
from nilearn.datasets import fetch_icbm152_brain_gm_mask  # load_mni152_brain_mask
from nilearn.input_data import NiftiLabelsMasker # NiftiMasker
from nilearn.image import load_img
from nilearn.plotting import view_img
from nilearn.regions import signals_to_img_labels
import wget

In [3]:
data = nilearn.datasets.fetch_neurovault_ids(collection_ids=[collection_id], data_dir=data_dir,mode='download_new')

Reading local neurovault data.
Already fetched 1 image
Already fetched 2 images
Already fetched 3 images
Already fetched 4 images
Already fetched 5 images
Already fetched 6 images
Already fetched 7 images
Already fetched 8 images
Already fetched 9 images
Already fetched 10 images
Already fetched 11 images
Already fetched 12 images
Already fetched 13 images
Already fetched 14 images
Already fetched 15 images
Already fetched 16 images
Already fetched 17 images
Already fetched 18 images
Already fetched 19 images
Already fetched 20 images
Already fetched 21 images
Already fetched 22 images
Already fetched 23 images
Already fetched 24 images
Already fetched 25 images
Already fetched 26 images
Already fetched 27 images
Already fetched 28 images
Already fetched 29 images
Already fetched 30 images
Already fetched 31 images
Already fetched 32 images
Already fetched 33 images
Already fetched 34 images
Already fetched 35 images
Already fetched 36 images
Already fetched 37 images
Already fetched 3

# 2. Select the data of interest for our study.

Each class is a condition. The data samples are contrast maps showing the effect of a condition on brain activity with respect to a baseline activity. In this section, we:
- retrieve all classes from the collection of NeuroVault,
- look at the number of data samples per class.


In [4]:
# The collection of Neurovault contains many contrast maps (data samples).
# Some of them show the effect of a condition over a baseline 
# (elementary contrast maps). Others represent more complex relationships such as 
# the effect of a condition over another condition.
# Not to include unwanted bias, we only kept the elementary contrast maps in the benchmark.
# In NeuroVault, their names look like "condition vs baseline" or only "condition".

# Additionally, we removed the two classes studied in the 'bang' task,
# as only 11 data samples are available per class.

# Create a list containing the name of all classes.
classes = set()

# We go through all contrast maps from the NeuroVault collection.
for image in data.images_meta:
    # We retrieve the name of the contrast.
    contrast_definition = image['contrast_definition']
    classes.add(image['contrast_definition'])                
classes = list(classes)

In [5]:
df_4337 = pd.DataFrame(columns=['task','name','contrast_definition'])
for image in data.images_meta:
    df_4337 = df_4337.append({'task':image['task'],'name':image['name'],
                              'contrast_definition':image['contrast_definition']},ignore_index=True)
df_4337['subject_id']=df_4337.name.apply(lambda x :x.split('_')[0])

In [6]:
df_4337[df_4337.contrast_definition.isin(classes)].groupby(['task','contrast_definition']).agg({'subject_id':'nunique'}).rename(columns={'subject_id':'number_of_subject'})

number_of_subject
task       contrast_definition                   
EMOTION    FACES                              786
           SHAPES                             786
GAMBLING   PUNISH                             785
           REWARD                             787
LANGUAGE   MATH                               786
           STORY                              786
MOTOR      CUE                                786
           LF                                 783
           LH                                 786
           RF                                 785
           RH                                 786
RELATIONAL MATCH                              785
           REL                                785
SOCIAL     RANDOM                             785
           TOM                                783
WM         0BK_BODY                           785
           0BK_FACE                           785
           0BK_PLACE                          785
           0BK_TOOL                           781
           2BK_BODY                           785
           2BK_FACE                           783
           2BK_PLACE                          784
           2BK_TOOL                           784

In the IBC dataset, data were collected with at least two acquisitions with opposite phase-encoding directions (AP or PA) per task and per subject. Statistics of their joint effects were calculated under a Fixed-Effects (FFX) model. As in machine learning the more samples we use the more performant the model will be, we chose to use the contrasts generated from the AP or PA instead of the ones calculated from a FFX model.

In [7]:
# Number of data samples per class.
examples = {}
[examples.setdefault(c, 0) for c in classes]

# We go through all contrast maps from the NeuroVault collection.
for image in data.images_meta:
    # We retrieve the name of the contrast.
    contrast_definition = image['contrast_definition']
    # We only keep the AP / PA contrasts.
    if contrast_definition in classes:
        name = image['name'].split('_')
        if 'ffx' not in name:
            examples[contrast_definition] += 1

print('min:', min(examples.values()), '\nQ1:', np.percentile(list(examples.values()), 25, interpolation = 'midpoint'), '\nmedian:', np.median(list(examples.values())), '\nQ3:', np.percentile(list(examples.values()), 75, interpolation = 'midpoint') , '\nmax:', max(examples.values()))
print("The total number of data samples in our study is", np.sum(list(examples.values())), '.')

min: 781 
Q1: 784.5 
median: 785.0 
Q3: 786.0 
max: 787
The total number of data samples in our study is 18052 .


In [8]:
# Number of examples per class and per task.
# Retrieve the names of the tasks.
tasks = []
for image in data.images_meta:
    if image['task'] not in tasks:
        tasks.append(image['task'])

# Retrieve to which task a condition is associated with.
contrasts = {}
[contrasts.setdefault(task, []) for task in tasks]

for image in data.images_meta:
    if image['contrast_definition'] not in contrasts[image['task']]:
        contrasts[image['task']].append(image['contrast_definition'])

# Per task, retrieve the conditions and the number of examples for each condition.
examples_per_task = {}
[examples_per_task.setdefault(task, []) for task in tasks]
actual_tasks = []
for condition in examples.keys():
    # Retrieve the task associated to the condition
    for key in contrasts.keys():
        if condition in contrasts[key]:
            task = key
            if task not in actual_tasks:
                actual_tasks.append(task)
            break
    examples_per_task[task].append({condition: examples[condition]})

# For instance:
TASK = 'MOTOR'
examples_per_task[TASK]

[{'LH': 786}, {'CUE': 786}, {'RH': 786}, {'LF': 783}, {'RF': 785}]

# 3. Create a dataloader.

## a. Data of interest
First, we need to store in a class the paths of the data samples and their labels.

In [9]:
def find(tasks:list,task_set:list):
    bin_tasks = []
    for t in tasks:
        if t in task_set:
            bin_tasks.append(True)
        else:
            bin_tasks.append(False)        
    return bin_tasks

class HCP():
    # This class aims to search images of subject(s) / task(s) / contrast(s) (or condition(s)).
    
    def __init__(self, ibc_data):
        # Get the lists of subject names, tasks, contrasts (and conditions) and images.
        # ibc_data : dictionnary obtained by using nilearn fetcher.
        
        self.subjectnames = [image['name'].split('_')[0] for image in ibc_data.images_meta]
        self.tasks = [image['task'] for image in ibc_data.images_meta]
        self.contrasts = [image['contrast_definition'] for image in ibc_data.images_meta]
        self.images = ibc_data.images
        self.meta = ibc_data.images_meta
        self.ffx = ['ffx' not in image['name'].split('_') for image in ibc_data.images_meta]

    def __getsourcetarget__(self, task_set:list, subject_set:list, contrast_set:list, remove_ffx=False):
        # 1) Select the images whose task is in task_set.
        # 2) Select the images whose subject is in subject_set.
        # 3) Select the images whose contrast in in contrast_set.
        # 4) We only select the ap/pa statistic_map (we do not consider the Fixed Effects maps).
        
        # Select the images associated to tasks.
        bin_tasks = find(self.tasks, task_set)
        subjectnames = list(compress(self.subjectnames, bin_tasks))
        tasks = list(compress(self.tasks, bin_tasks))
        contrasts = list(compress(self.contrasts, bin_tasks))
        images = list(compress(self.images, bin_tasks))
        meta = list(compress(self.meta, bin_tasks))
        ffx = list(compress(self.ffx, bin_tasks))

        # Select the images associated subjects.
        bin_subjectnames = find(subjectnames, subject_set)
        subjectnames = list(compress(subjectnames, bin_subjectnames))
        tasks = list(compress(tasks, bin_subjectnames))
        contrasts = list(compress(contrasts, bin_subjectnames))
        images = list(compress(images, bin_subjectnames))
        meta = list(compress(meta, bin_subjectnames))
        ffx = list(compress(ffx, bin_subjectnames))

        # Select the images associated to contrasts.
        bin_contrasts = find(contrasts, contrast_set)
        subjectnames = list(compress(subjectnames, bin_contrasts))
        tasks = list(compress(tasks, bin_contrasts))
        contrasts = list(compress(contrasts, bin_contrasts))
        images = list(compress(images, bin_contrasts))
        meta = list(compress(meta, bin_contrasts))
        ffx = list(compress(ffx, bin_contrasts))
        
        # Keep the ap/pa statistic_map only.
        if remove_ffx:
            subjectnames = list(compress(subjectnames, ffx))
            tasks = list(compress(tasks, ffx))
            contrasts = list(compress(contrasts, ffx))
            images = list(compress(images, ffx))
            meta = list(compress(meta, ffx))

        return images, contrasts, tasks, subjectnames, meta

dataset = HCP(data)

In [10]:
# images contains the paths to all nii elementary contrast associated with a ap/pa direction (not ffx).
# labels contains the name of the associated classes.
# tasks, subjectnames and meta contain extra-information.
images, labels, tasks, subjectnames, meta = dataset.__getsourcetarget__(np.unique(dataset.tasks), np.unique(dataset.subjectnames), classes)

## b. Parcellation
We can either use the full contrast maps or parcel them into bigger regions before using them. To that end, we compute all parcellations and store them.

In [11]:
# MNI 152 is a 3D coordinate system for localization in the brain. It is an average taken from 152 healthy individuals.
# ICBM452 -> ICBM152 -> MNI152 -> MNI305 -> Talairach -> Brodmann.
# Current standard MNI template: ICBM152. It is an average of 152 normal MRI scans that have been matched to the MNI305.
mask_mni = fetch_icbm152_brain_gm_mask()

# Glasser is a reference map for regions in the brain. It splits the brain into 360 regions.
glassermasker = NiftiLabelsMasker(labels_img=path_Glasser,mask_img=mask_mni)
glassermasker.fit()

for curmeta in tqdm(meta):
    # Location of the MRI scan on NeuroVault.
    current_url = curmeta['file']
    # Name attributed to the image locally.
    current_newname = os.path.split(curmeta['relative_path'])[1]
    filepath = os.path.join(os.path.join(data_dir, 'neurovault/collection_'+str(collection_id)+'/'), current_newname)


    # Some images were missing. So we manually downloaded them.
    if os.path.exists(filepath):
        #print("File {} already exists".format(filepath))
        pass
    else:
        #print("Downloading file {}...".format(filepath))
        wget.download(url=current_url, out = filepath)
    # Attribute to each image nii a parcellated version stored in a npz file.
    npzfile = os.path.splitext(os.path.splitext(filepath)[0])[0]+'.npz'
    temp_path = os.path.join(os.path.join(data_dir, 'neurovault/collection_'+str(collection_id)+'/npz_files_'+str(collection_id)), current_newname)
    npzfile_second_path = os.path.splitext(os.path.splitext(temp_path)[0])[0]+'.npz'
    if os.path.exists(npzfile):
        #print("Parcellated file {} already exists".format(npzfile))
        pass
    else:       
        #print("Parcellating...")
        X = glassermasker.fit_transform([filepath])
        ## Save the npz file
        savez_compressed(npzfile,X=X)
        savez_compressed(npzfile_second_path,X=X)

    if os.path.exists(npzfile_second_path):
        #print("Parcellated file {} already exists".format(npzfile))
        pass
    else:       
        #print("Parcellating...")
        X = glassermasker.fit_transform([filepath])
        ## Save the npz file
        savez_compressed(npzfile_second_path,X=X)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 18052/18052 [00:00<00:00, 62790.84it/s]


## d. Splitting the dataset into three datasets: base / validation / novel.

To create a benchmark dataset for few-shot learning, we need to split the classes into three sets: base / validation / novel. 

The first two are used to extract general knowledge. The last one to mimic studies with few examples. In our github, we already provide the splits we used in our article.

If you want to generate a new split, you can follow our code and uncomment the last three lines.

In [12]:
df_gby = df_4337.groupby('contrast_definition').agg({'subject_id':[set,'nunique']})['subject_id'].reset_index()

In [16]:
df_ = df_gby[df_gby['nunique']!=df_gby['nunique'].max()].reset_index()

In [32]:
set_subjects = df_.loc[0]['set']
for i in range(len(df_)-1):
    set_subjects = set_subjects.intersection(df_.loc[i+1]['set'])  

In [33]:
print('at least '+str(len(list(set_subjects)))+' common subjects per task')

at least 764 common subjects per task


In [42]:
np.random.seed(10)
id_train = list(np.random.choice(np.array(list(set(set_subjects))),round(70/100*len(df_4337.subject_id.unique())),replace=False))
id_val = [x for x in list(set_subjects) if x not in id_train]

In [77]:
nb_total_subject = len(df_4337.subject_id.unique())
np.random.seed(10)
id_train = list(np.random.choice(np.array(list(set(set_subjects))),round(70/100*nb_total_subject),replace=False))
id_val_temp = [x for x in list(set_subjects) if x not in id_train]
id_val = list(np.random.choice(np.array(id_val_temp),round(15/100*nb_total_subject),replace=False))
id_test = [x for x in list(set(subjectnames)) if x not in id_train+id_val]

In [78]:
df = pd.DataFrame(list(zip(images, labels, tasks, meta,subjectnames)),columns=['Filename','Label','Task','Meta','Subject_id'])
df['Filename'] = df.Filename.apply(lambda x: x.split('/')[-1].split('.')[0])
df_train = df[df['Subject_id'].isin(id_train)]
df_val = df[df['Subject_id'].isin(id_val)]
df_test = df[df['Subject_id'].isin(id_test)]

print('The base set contains ', df_train.Label.nunique(), 'classes.')
print('The validation set contains ', df_val.Label.nunique(), 'classes.')
print('The test set contains ', df_test.Label.nunique(), 'classes.')
print('The base set contains ', len(df_train), 'data samples.')
print('The validation set contains ', len(df_val), 'data samples.')
print('The test set contains ', len(df_test), 'data samples.')

The base set contains  23 classes.
The validation set contains  23 classes.
The test set contains  23 classes.
The base set contains  12673 data samples.
The validation set contains  2714 data samples.
The test set contains  2665 data samples.


In [79]:
if not os.path.exists('splits/split_HCP'):
    os.makedirs('splits/split_HCP')
df_train.to_csv('splits/split_HCP/'+'train.csv',index=False)
df_val.to_csv('splits/split_HCP/'+'val.csv',index=False)
df_test.to_csv('splits/split_HCP/'+'test.csv',index=False)